In [1]:
from pathlib import Path
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt
from pyometiff import OMETIFFWriter
import webknossos as wk

import dask.array as da

import pandas as pd
from skimage.measure import label, regionprops_table

In [2]:
# this comes from the WEBKNOSSOS website, under the user settings
AUTH_TOKEN = "1mng65J7d-5IVFmfJoF4rw" # from gisela
WK_TIMEOUT="3600" # in seconds
ORG_ID = "83d574429f8bc523" # gisela's webknossos
dsets = ("60_2_6R_crop1", "GN60_2_5R_crop1", "60_2_2L_crop1_")
DATASET_1 = "60_2_5R"
DATASET_2 = "60_2_3L"
Annotation = namedtuple("Annotation", ["ID", "Dataset", "Name"])
sample1   = Annotation("6644c04d0100004a01fa11af", DATASET_1,"60_2_5R")
sample2    = Annotation("664316880100008a049e890e", DATASET_2,"60_2_3L")

In [3]:
# the dataset url comes from the WEBKNOSSOS website, open the image of interest from the dashboard and check
# I removed the view information

ANNOTATION_ID = sample1.ID
DATASET_NAME = sample1.Dataset
with wk.webknossos_context(token=AUTH_TOKEN):
    ds = wk.Dataset.open_remote(dataset_name_or_url=DATASET_NAME, organization_id=ORG_ID)
    img_layer = ds.get_color_layers()
    assert len(img_layer) == 1, "more than an image, this is unexpected for this project"
    img_layer = img_layer[0]
    annotations = wk.Annotation.open_as_remote_dataset(annotation_id_or_url=ANNOTATION_ID)
    lbl_layers = annotations.get_segmentation_layers()
    #dataset = wk.Dataset.open_remote(DATASET_URL)
    #img_data = dataset.get_color_layer().get_finest_mag().read()

c:\Users\rafaelCCI\mambaforge\envs\COMULIS\lib\site-packages\webknossos\annotation\annotation.py:455: UserWarning: [INFO] Omitting the volume layer annotation data for layers ['Myelin', 'Mitochondria', 'Dystrophic_myelin', 'Axon'], as their location is not referenced in the NML.
  warnings.warn(


In [4]:
Pixel_size = namedtuple("Pixel_size_nm", ["x", "y", "z", "MAG", "unit"])

voxel_size = ds.voxel_size
print(img_layer)
print(f'image size in pixels {img_layer.bounding_box.size.to_np()}')
print(f'voxel size in nm: {ds.voxel_size}')

mag_list = list(img_layer.mags.keys())
print(mag_list)
MAG = mag_list[1]
pSize = Pixel_size(voxel_size[0] * MAG.x, voxel_size[1] * MAG.y, voxel_size[2] * MAG.z, MAG=MAG, unit="nm")
pSize


Layer('60_2_5R', dtype_per_channel=uint8, num_channels=1)
image size in pixels [24789 23068     1]
voxel size in nm: (5.0, 5.0, 70.0)
[Mag(1), Mag(2-2-1), Mag(4-4-1), Mag(8-8-1), Mag(16-16-1), Mag(32-32-1), Mag(64-64-1), Mag(128-128-1), Mag(256-256-1)]


Pixel_size_nm(x=10.0, y=10.0, z=70.0, MAG=Mag(2-2-1), unit='nm')

In [5]:
MAG = mag_list[4]
pSize = Pixel_size(voxel_size[0] * MAG.x, voxel_size[1] * MAG.y, voxel_size[2] * MAG.z, MAG=MAG, unit="nm")
print(pSize)
with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    img_data = img_layer.get_mag(pSize.MAG).read()
print(img_data.shape)
img_dask = da.from_array(np.swapaxes(img_data,-1,-3), chunks=(1,1,512,512))
img_dask

Pixel_size_nm(x=80.0, y=80.0, z=70.0, MAG=Mag(16-16-1), unit='nm')
(1, 1550, 1442, 1)


dask.array<array, shape=(1, 1, 1442, 1550), dtype=uint8, chunksize=(1, 1, 512, 512), chunktype=numpy.ndarray>

In [6]:
output_fpath = Path.cwd().joinpath("gisela_ds_" + img_layer.name + "_mag_" + str(pSize.MAG) + ".ome.tiff")
print(output_fpath)

assert pSize.unit == "nm", "should be nm for this to work"
metadata_dict = {
    "PhysicalSizeX" : str(pSize.x/1000),
    "PhysicalSizeXUnit" : "µm",
    "PhysicalSizeY" : str(pSize.y/1000),
    "PhysicalSizeYUnit" : "µm",
    "PhysicalSizeZ" : str(pSize.z/1000),
    "PhysicalSizeZUnit" : "µm",
    "Channels" : {
        "SEM" : {
            "Name" : "BSD",
            "SamplesPerPixel": 1,
        },
    }
}

# a string describing the dimension ordering
dimension_order = "CZYX"

writer = OMETIFFWriter(
    fpath=output_fpath,
    dimension_order=dimension_order,
    array=img_dask,
    metadata=metadata_dict,
    explicit_tiffdata=False)

writer.write()

e:\PROJECTS\CCI\Gisela\gisela_ds_60_2_5R_mag_16-16-1.ome.tiff


In [15]:
for lbl in lbl_layers:
    print(lbl)
    print(lbl.name)
    print(lbl.bounding_box.size.to_np())

Layer('Myelin', dtype_per_channel=uint32, num_channels=1)
Myelin
[24789 23068     1]
Layer('Mitochondria', dtype_per_channel=uint32, num_channels=1)
Mitochondria
[24789 23068     1]
Layer('Dystrophic_myelin', dtype_per_channel=uint32, num_channels=1)
Dystrophic_myelin
[24789 23068     1]
Layer('Axon', dtype_per_channel=uint32, num_channels=1)
Axon
[24789 23068     1]


In [25]:
annotation_name = "Myelin"
annotation_idx = 0
for lbl in lbl_layers:
    if lbl.name == annotation_name:
        annotation_idx = annotation_idx
        break
    else:
        annotation_idx += 1

assert annotation_idx < len(lbl_layers), "cant find layer"
with wk.webknossos_context(token=AUTH_TOKEN):
    lbl_data = lbl_layers[annotation_idx].get_mag(pSize.MAG).read()
unique_lbls = np.unique(lbl_data)
print(unique_lbls)


if np.max(unique_lbls) < 512:
    lbl_data = lbl_data.astype(np.uint8)

lbl_dask = da.from_array(np.swapaxes(lbl_data,-1,-3), chunks=(1,5,512,512))
lbl_dask

0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50]


dask.array<array, shape=(1, 1, 1442, 1550), dtype=uint8, chunksize=(1, 1, 512, 512), chunktype=numpy.ndarray>

In [26]:
output_fpath = Path.cwd().joinpath("gisela_ds_" + img_layer.name + "_mag_" + str(pSize.MAG) + "_lbl_" + annotation_name + ".ome.tiff")
print(output_fpath)

assert pSize.unit == "nm", "should be nm for this to work"
metadata_dict = {
    "PhysicalSizeX" : str(pSize.x/1000),
    "PhysicalSizeXUnit" : "µm",
    "PhysicalSizeY" : str(pSize.y/1000),
    "PhysicalSizeYUnit" : "µm",
    "PhysicalSizeZ" : str(pSize.z/1000),
    "PhysicalSizeZUnit" : "µm",
    "Channels" : {
        "SEM" : {
            "Name" : annotation_name,
            "SamplesPerPixel": 1,
        },
    }
}

# a string describing the dimension ordering
dimension_order = "CZYX"

writer = OMETIFFWriter(
    fpath=output_fpath,
    dimension_order=dimension_order,
    array=lbl_dask,
    metadata=metadata_dict,
    explicit_tiffdata=False)

writer.write()

e:\PROJECTS\CCI\Gisela\gisela_ds_60_2_5R_mag_16-16-1_lbl_Myelin.ome.tiff


In [22]:
import napari

In [27]:
viewer = napari.view_image(data=img_dask,channel_axis=0,name=img_layer.name)
viewer.add_labels(lbl_dask, name=annotation_name)

<Labels layer 'Myelin' at 0x1e2b71d0f10>

In [ ]:
# run connected component analysis
#lbl = label(mask)

properties = ['label', 'area','eccentricity','solidity','intensity_mean',
              'bbox','axis_minor_length','axis_major_length','feret_diameter_max']
table = regionprops_table(label_image=seg.squeeze(),
                          intensity_image=img_data.squeeze(),
                          properties=properties)
table = pd.DataFrame(table)
print(table.head())
print(table.describe())
table.hist(column='area', figsize=(4, 4))

In [ ]:
n_neu = table.shape[0]
print(f"number of segmented objects: {n_neu}")

In [ ]:
from skimage.morphology import medial_axis, binary_opening
from skimage.morphology import disk
from functools import partial

def get_roi(table_col, delta=50):
    minr = int(table_col['bbox-0']-delta)
    maxr = int(table_col['bbox-2']+delta)
    minc = int(table_col['bbox-1']-delta)
    maxc = int(table_col['bbox-3']+delta)

    return (minr, maxr, minc, maxc)

def get_AR(table_col):
    # simple calc
    return table_col['axis_major_length']/table_col['axis_minor_length']

def get_BW_im(table_col, raw_data, seg):
    lbl_id = table_col['label']
    idx_roi = get_roi(table_col)

    im  = raw_data.squeeze()[idx_roi[0]:idx_roi[1],idx_roi[2]:idx_roi[3]]
    lbl =  seg.squeeze()[idx_roi[0]:idx_roi[1],idx_roi[2]:idx_roi[3]]
    BW = lbl==lbl_id

    return BW, im

def get_width(BW, disk_diam=5):
    # simple calculation of with based of skeleton and distance transform
    footprint = disk(disk_diam)
    opened = binary_opening(BW, footprint)
    # Compute the medial axis (skeleton) and the distance transform
    skel, distance = medial_axis(opened, return_distance=True)

    # Distance to the background for pixels of the skeleton
    dist_on_skel = distance * skel
    w = dist_on_skel[dist_on_skel>0]

    return np.median(w)*2

def get_width_table(table_col, raw_data, seg, diam):

    BW, im = get_BW_im(table_col, raw_data, seg)

    myalin_w = get_width(BW, disk_diam=diam)

    return myalin_w

In [ ]:
table['AR'] = table.apply(get_AR, axis='columns',result_type='expand')
#table[['minr', 'maxr', 'minc', 'maxc']] = table.apply(partial(get_roi, delta=50), axis='columns',result_type='expand')
table['width'] = table.apply(partial(get_width_table, raw_data=img_data, seg=seg, diam=5), axis='columns',result_type='expand')
table['Image_name'] = current_dset[2]
table.sample(5)

In [ ]:
table.to_csv(current_dset[2] + "_morphometrics.csv", sep='\t')
table.to_excel(current_dset[2] + "_morphometrics.xlsx", sheet_name=current_dset[2])